In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [2]:

%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm

    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [5]:
from unsloth import FastModel
import torch
max_seq_length = 1024

fourbit_models = [

    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
]

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = max_seq_length,
    load_in_4bit = False,
    load_in_8bit = False,
    full_finetuning = False,

)

==((====))==  Unsloth 2025.3.18: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [6]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,

    r = 8,
    lora_alpha = 8,
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [7]:
from datasets import load_dataset
dataset = load_dataset("openai/gsm8k", "main", split = "train")
dataset

README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer'],
    num_rows: 7473
})

In [8]:
def extract_answer(text):
  if '####' not in text: return None
  return text.split("####")[1].strip()
extract_answer(dataset[0]["answer"])


'72'

In [9]:
reasoning_start = "<pensando>"
reasoning_end = "</pensando>"

solution_start = "<Solucion>"
solution_end = "</Solucion>"

system_prompt = \
f""" Te presentan un problema.
piensa en el mismo y divide al mismo en pasos.
tus pasos deben ir dentro de {reasoning_start} y {reasoning_end}.
luego debes colocar la solucion entre: {solution_start} y {solution_end}
"""

In [10]:
dataset = dataset.map(lambda x:{
    "prompt":[
        {"role":"system","content":system_prompt},
        {"role":"user","content":x["question"]}
    ],
    "answer":extract_answer(x["answer"])
})

dataset[0]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': '72',
 'prompt': [{'content': ' Te presentan un problema. \npiensa en el mismo y divide al mismo en pasos.\ntus pasos deben ir dentro de <pensando> y </pensando>.\nluego debes colocar la solucion entre: <Solucion> y </Solucion>\n',
   'role': 'system'},
  {'content': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
   'role': 'user'}]}

In [33]:
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning_start}.+?{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

In [34]:
match_format.search(
    "<start_working_out>Let me think!<end_working_out>"\
    "<SOLUTION>2</SOLUTION>",
)

In [35]:
def match_format_exactly(completions,**kwargs):
  scores = []
  for completion in completions:
    score = 0
    response = completion[0]["content"]
    if match_format.search(response) is not None:
      score += 3.0
    scores.append(score)
  return scores


In [36]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        score += 0.5 if response.count(reasoning_start) == 1 else -0.5
        score += 0.5 if response.count(reasoning_end)   == 1 else -0.5
        score += 0.5 if response.count(solution_start)  == 1 else -0.5
        score += 0.5 if response.count(solution_end)    == 1 else -0.5
        scores.append(score)
    return scores

In [37]:
def check_answer(prompts,completions,answer,**kwargs):
  question = prompts[0][-1]["content"]
  responses = [completion[0]["content"] for completion in completions]

  extracted_responses = [
      guess.group(1)
      if(guess := match_format.search(response)) is not None
      else None
      for response in responses
  ]

  scores = []

  for guess,true_answer in zip(extracted_responses,answer):
    score = 0
    if guess is None:
      scores.append(0)
      continue
    if guess == true_answer:
      score += 3.0
    elif guess.strip() == true_answer.strip():
      score += 1.5

    else:
      try:
        ratio = float(guess) / float(true_answer)
        if ratio >= 0.9 and ratio <= 1.1: score += 0.5
        elif ratio >= 0.8 and rtio <= 1.2: score += 0.25
        else:score -= 1.0
      except:
        score -= 0.5
    scores.append(score)
  return scores


In [38]:
match_numbers = re.compile(
    rf"{solution_start}.*?([\d\.]{{1,}})",
    flags = re.MULTILINE | re.DOTALL
)


In [42]:
def check_numbers(prompts,completions,answer,**kwargs):
  question = prompts[0][-1]["content"]
  responses = [completion[0]["content"] for completion in completions]

  extracted_responses = [
      guess.group(1)
      if (guess := match_numbers.search(r)) is not None else None\
      for r in responses
  ]
  scores = []
  print('*'*20, f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
  for guess,true_answer in zip(extracted_responses,answer):
    if guess is None:
      scores.append(0)
      continue
    try:
      true_answer = float(true_answer.strip())
      guess = float(guess.strip())
      scores.append(1.5 if guess == true_answer else 0.0)
    except:
      scores.append(0)
  return scores



In [43]:
max_prompt_length = 256

from trl import GRPOConfig,GRPOTrainer

training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1,
    num_generations = 4,
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    max_steps = 50,
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "none",
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

Unsloth: Switching to float32 training since model cannot work with float16


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 6,522,880/1,006,408,832 (0.65% trained)


******************** Question:
A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all? 
Answer:
476 
Response:
<pensando>El problema presenta un descuento por cada ticket que exceda los 10. Necesitamos calcular el número de tickets que exceden los 10 y luego determinar el precio total del descuento. El precio original de cada ticket es $40. </pensando>
<pensando>Primero, necesitamos encontrar el número de tickets que exceden los 10.  12 tickets - 10 tickets = 2 tickets. </pensando>
<pensando>El descuento es del 5% de cada ticket que supera los 10.  El descuento en cada 2 tickets es 5% de $40, que es 0.05 * $40 = $2. </pensando>
<pensando>El precio total a pagar es el precio original de cada ticket menos el descuento.  Entonces, el precio total es $40 - $2 = $38. </pensando>
<Solucion>38</Solucion> 
Extracted:
38


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / match_format_exactly,rewards / match_format_approximately,rewards / check_answer,rewards / check_numbers
1,-0.000000,2.125000,1.652019,235.250000,0.000000,1.500000,1.000000,-0.375000,0.000000
2,0.000000,1.000000,3.027650,369.750000,0.000000,1.500000,-0.250000,-0.250000,0.000000
3,0.000000,2.125000,1.887459,387.000000,0.000007,0.750000,0.750000,-0.125000,0.750000
4,0.000000,0.750000,1.554563,223.500000,0.000014,0.750000,-0.250000,-0.125000,0.375000
5,0.000000,3.375000,2.250000,137.250000,0.000070,2.250000,1.500000,-0.375000,0.000000
6,0.000000,2.375000,2.250000,438.000000,0.000140,2.250000,0.250000,-0.125000,0.000000
7,0.000000,-0.500000,0.577350,491.750000,0.000013,0.000000,-0.500000,0.000000,0.000000
8,0.000000,0.875000,2.462214,117.500000,0.000054,0.750000,0.250000,-0.125000,0.000000
9,0.000000,1.250000,1.500000,304.750000,0.000043,2.250000,-0.250000,-0.750000,0.000000
10,0.000000,0.750000,1.190238,395.250000,0.000025,0.750000,-0.250000,-0.125000,0.375000


Unsloth: Will smartly offload gradients to save VRAM!
******************** Question:
Jane is trying to decide whether to buy a house or a trailer. A house costs $480,000 and a trailer costs $120,000. Each loan will be paid in monthly installments over 20 years. How much more is the monthly payment on the house compared to the trailer? 
Answer:
1500 
Response:
<pensando>Este es un problema de comparación de costos. Necesitamos encontrar la diferencia en los pagos mensuales para determinar cuánto más cuesta comprar una casa en comparación con una casa de enganche en lugar de una casa de enganche. Necesitaré calcular los pagos mensuales de cada opción y luego comparar los resultados. La información que tengo es el costo inicial de la casa y el costo inicial de la casa de enganche, así como la duración del préstamo. Necesito encontrar la diferencia entre los pagos mensuales de cada opción.</pensando>
<pensando>Primero, calcularemos el costo total de la casa. El costo total es $480,000. Lue

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    tokenize = False,
)
from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 64,
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)